## Ref

https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/5-Fine%20Tuning/Prompt_Tuning_PEFT.ipynb

https://medium.com/@anyuanay/fine-tuning-the-pre-trained-t5-small-model-in-hugging-face-for-text-summarization-3d48eb3c4360

https://stackoverflow.com/questions/75854700/how-to-fine-tune-a-huggingface-seq2seq-model-with-a-dataset-from-the-hub

## download the library:

In [ ]:
pip install transformers datasets torch peft accelerate evaluate rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   

## download the dataset

download the dataset that we make by ourself and upload it on the hugging face:

Split 90% for training, 10% for validation due to don't have much len of datset (10K)so we use 90% for trainnig it and 10% for validation:

In [ ]:
from datasets import load_dataset, DatasetDict

# Load dataset
dataset = load_dataset("kritsadaK/EDGAR-CORPUS-Financial-Summarization")

# Split 90% for training, 10% for validation
split_dataset = dataset["train"].train_test_split(test_size=0.1)

# Rename test set to validation
dataset = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

EDGAR-CORPUS-Financial-Summarization.csv:   0%|          | 0.00/794M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10610 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'summary', 'model'],
        num_rows: 9549
    })
    validation: Dataset({
        features: ['input', 'summary', 'model'],
        num_rows: 1061
    })
})


## download the model:

we used bart - facebook for fine tuning our dataset

In [ ]:
from transformers import AutoTokenizer

model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    model_inputs = tokenizer(examples["input"], max_length=1024, truncation=True)
    labels = tokenizer(examples["summary"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/9549 [00:00<?, ? examples/s]

Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

## loading a pre-trained model

loads a pre-trained sequence-to-sequence model specified by model_name.

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

## setting the parameter:

https://stackoverflow.com/questions/75854700/how-to-fine-tune-a-huggingface-seq2seq-model-with-a-dataset-from-the-hub

https://medium.com/@anyuanay/fine-tuning-the-pre-trained-t5-small-model-in-hugging-face-for-text-summarization-3d48eb3c4360

Explanation of each parameter:
- `output_dir`: Directory to save the model checkpoints.
- `evaluation_strategy`: Evaluate the model after every epoch.
- `save_strategy`: Save the model checkpoint after every epoch.
- `learning_rate`: Learning rate for training.
- `per_device_train_batch_size`: Batch size for training on each device.
- `per_device_eval_batch_size`: Batch size for evaluation on each device.
- `weight_decay`: Regularization technique to prevent overfitting.
- `save_total_limit`: Maximum number of saved model checkpoints.
- `num_train_epochs`: Number of training epochs.
- `predict_with_generate`: Enables text generation during evaluation.
- `fp16`: Enables mixed precision training for faster computation (if using a GPU).
- `logging_dir`: Directory to save the training logs.

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-summarization",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,  # Enable mixed precision for speedup (if using GPU)
    logging_dir="./logs"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


sets up the data collator to prepare batches of input data for training the sequence-to-sequence model.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Seq2SeqTrainer instance by passing the training arguments along with the model, dataset, tokenizer, data collator, and the compute_metrics function:

trainer = Seq2SeqTrainer(...):
* model=model: Specifies the pre-trained sequence-to-sequence model to be used.
* args=training_args: Specifies the training arguments configured previously.
* train_dataset=tokenized_dataset["train"]: Specifies the tokenized dataset for training.
* eval_dataset=tokenized_dataset["validation"]: Specifies the tokenized dataset for evaluation.
* tokenizer=tokenizer: Specifies the tokenizer used for preprocessing the input data.
* data_collator=data_collator: Specifies the data collator to prepare batches of input data.

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


<ipython-input-7-934a1e2740a7>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


log the loss of each epoch:

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kritsada8774 (kritsada8774-assumption-university-of-thailand) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,1.385300,1.315511
2,1.166400,1.209022
3,1.071600,1.179914


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=14325, training_loss=1.2769778373145724, metrics={'train_runtime': 2082.7901, 'train_samples_per_second': 13.754, 'train_steps_per_second': 6.878, 'total_flos': 6.208104553670246e+16, 'train_loss': 1.2769778373145724, 'epoch': 3.0})

https://medium.com/@eren9677/text-summarization-387836c9e178

https://www.geeksforgeeks.org/understanding-bleu-and-rouge-score-for-nlp-evaluation/

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used to evaluate the quality of text generation, such as summarization and translation, by comparing the generated text with reference texts. ROUGE measures the overlap between the generated text and the reference texts, focusing on different aspects of the text.

ROUGE scores are usually reported as precision, recall, and F1-score (a harmonic mean of precision and recall). These scores help assess how well the generated text matches the reference texts in terms of content and structure.

* preds, labels = eval_pred: Unpacks the predictions and labels from the evaluation predictions.
* decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True): Decodes the predicted sequences.
* decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True): Decodes the reference sequences (labels).
* result = rouge.compute(predictions=decoded_preds, references=decoded_labels): Computes the ROUGE scores for the predictions and references.
return {k: v.mid.fmeasure for k, v in result.items()}: Returns the F1 scores (f-measure) for each ROUGE score.

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: v.mid.fmeasure for k, v in result.items()}

# Run evaluation
trainer.evaluate()


{'eval_loss': 1.1799135208129883,
 'eval_runtime': 18.9002,
 'eval_samples_per_second': 56.137,
 'eval_steps_per_second': 28.095,
 'epoch': 3.0}

## save the model

In [ ]:
trainer.save_model("./bart-summarization-finetuned")
tokenizer.save_pretrained("./bart-summarization-finetuned")


('./bart-summarization-finetuned/tokenizer_config.json',
 './bart-summarization-finetuned/special_tokens_map.json',
 './bart-summarization-finetuned/vocab.json',
 './bart-summarization-finetuned/merges.txt',
 './bart-summarization-finetuned/added_tokens.json',
 './bart-summarization-finetuned/tokenizer.json')

upload the model on the hugging_face_model on the our repo:

In [ ]:
from huggingface_hub import HfApi, HfFolder

# Define repository and token
repo_name = "kritsadaK/bart-financial-summarization"
token = "secret"

# Authenticate
api = HfApi()
HfFolder.save_token(token)

# Model Path
local_model_path = "/content/bart-summarization-finetuned"

# Push the model to the existing repository
api.upload_folder(repo_id=repo_name, folder_path=local_model_path, path_in_repo="")

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kritsadaK/bart-financial-summarization/commit/ed0401dc7a5f1a0b0da3ccde5078692b8969acb6', commit_message='Upload folder using huggingface_hub', commit_description='', oid='ed0401dc7a5f1a0b0da3ccde5078692b8969acb6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kritsadaK/bart-financial-summarization', endpoint='https://huggingface.co', repo_type='model', repo_id='kritsadaK/bart-financial-summarization'), pr_revision=None, pr_num=None)